# Crypto Portfolio Optimization

Goal: Create a machine learning-driven portfolio optimization system for cryptocurrencies, dynamically adjusting allocations to maximize returns and minimize risks based on predictive analytics.

### Top crypto cryptocurrencies 

In [1]:
import requests
import pandas as pd

##### Coingecko API to get the top 20 cryptos 

In [2]:
def fetch_top_coins_tickers(vs_currency='usd', limit=30):
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&ids&x_cg_demo_api_key=CG-RJK8hmewzGZMgR1PHncrkr84'
    params = {
        'vs_currency': vs_currency,
        'order': 'market_cap_desc',
        'per_page': limit,
        'page': 1,
        'sparkline': False,
        'price_change_percentage': '24h'
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Error fetching data: Status code {response.status_code}")
        return []

    data = response.json()

##Extract tickers
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

In [3]:
top_coins = fetch_top_coins_tickers(vs_currency='usd', limit=20)
print(top_coins)

['BTC', 'ETH', 'USDT', 'XRP', 'BNB', 'SOL', 'USDC', 'ADA', 'DOGE', 'TRX', 'STETH', 'WBTC', 'LINK', 'TON', 'LEO', 'WSTETH', 'XLM', 'AVAX', 'USDS', 'HBAR']


##### yfinance (historical data)

In [8]:
import yfinance as yf


# Specify the symbols for the cryptocurrencies you are interested in
symbols = [f"{coin}-USD" for coin in top_coins]  


df = pd.DataFrame()

# Specify the date range
start_date = '2011-9-1'
end_date = '2024-12-17'

# Fetch the historical data for each symbol
for symbol in symbols:
    print(f"Fetching data for {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date)
    if not data.empty:
        data['Symbol'] = symbol.replace('-USD', '')  
        df = pd.concat([df, data], axis=0)


df.reset_index(inplace=True)




[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BTC-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ETH-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['USDT-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XRP-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BNB-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['SOL-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['USDC-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ADA-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*******************

Fetching data for BTC-USD...
Fetching data for ETH-USD...
Fetching data for USDT-USD...
Fetching data for XRP-USD...
Fetching data for BNB-USD...
Fetching data for SOL-USD...
Fetching data for USDC-USD...
Fetching data for ADA-USD...
Fetching data for DOGE-USD...
Fetching data for TRX-USD...
Fetching data for STETH-USD...
Fetching data for WBTC-USD...
Fetching data for LINK-USD...
Fetching data for TON-USD...




1 Failed download:
['TON-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Failed to get ticker 'LEO-USD' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['LEO-USD']: Exception('%ticker%: No timezone found, symbol may be delisted')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['WSTETH-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['XLM-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AVAX-USD']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')
Failed to get ticker 'USDS-USD' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%%**********************]  1 of 1 completed

1 Failed dow

Fetching data for LEO-USD...
Fetching data for WSTETH-USD...
Fetching data for XLM-USD...
Fetching data for AVAX-USD...
Fetching data for USDS-USD...
Fetching data for HBAR-USD...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   index   0 non-null      int64
dtypes: int64(1)
memory usage: 132.0 bytes


### Crypto History Data - data cleansing 

In [7]:
df

,index


In [6]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.rename(columns={'Volume': 'Volume($)'})

KeyError: 'Date'

In [7]:
df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume($)    0
Symbol       0
dtype: int64

In [8]:
df['Date'] = pd.to_datetime(df['Date'])

# Calculate daily returns for each cryptocurrency
df['Daily Return'] = df.groupby('Symbol')['Close'].pct_change()

In [9]:
df.isnull().sum()

Date              0
Open              0
High              0
Low               0
Close             0
Adj Close         0
Volume($)         0
Symbol            0
Daily Return    236
dtype: int64

In [10]:
df_filtered_date = df[df['Date'] == '2024-01-01']

df_sorted_by_close = df_filtered_date.sort_values(by='Close', ascending=False)

df_sorted_by_close[['Symbol', 'Close']]  

,Symbol,Close
3393,BTC,44167.332031
37575,WBTC,44080.980469
32298,WSTETH,2699.930176
5988,ETH,2352.327881
21797,STETH,2340.830322
13773,BNB,314.408295
15485,SOL,109.508682
28536,AVAX,41.864464
31131,LINK,15.535809
42560,DOT,8.598925


In [11]:
rows_with_null = df[df.isnull().any(axis=1)]

rows_with_null

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,BTC,NaN
3744,2017-11-09,308.644989,329.451996,307.056000,320.884003,320.884003,893249984,ETH,NaN
6339,2017-11-09,1.010870,1.013270,0.996515,1.008180,1.008180,358188000,USDT,NaN
8934,2017-11-09,0.217911,0.221791,0.214866,0.217488,0.217488,147916992,XRP,NaN
11529,2017-11-09,2.053140,2.174230,1.893940,1.990770,1.990770,19192200,BNB,NaN
...,...,...,...,...,...,...,...,...,...
34207,2020-08-29,6.934732,6.963617,6.834933,6.900954,6.900954,1222026,TON,NaN
35778,2019-01-30,3494.277588,3502.762939,3477.248047,3496.455322,3496.455322,1475,WBTC,NaN
37926,2022-03-18,0.009705,0.011553,0.009705,0.009937,0.009937,302479,SUI,NaN
38736,2017-11-09,0.033792,0.040380,0.033744,0.039946,0.039946,32813200,XLM,NaN


In [12]:
df['Symbol'].unique()

array(['BTC', 'ETH', 'USDT', 'XRP', 'BNB', 'SOL', 'DOGE', 'USDC', 'STETH',
       'ADA', 'TRX', 'AVAX', 'LINK', 'WSTETH', 'SHIB', 'TON', 'WBTC',
       'SUI', 'XLM', 'DOT'], dtype=object)

In [13]:

exclude_symbols = ['SHIB', 'ICP','STETH','WBTC','USDC','UNI']

# Filter 
df = df[~df['Symbol'].isin(exclude_symbols)]


In [14]:
df['Symbol'].unique()

array(['BTC', 'ETH', 'USDT', 'XRP', 'BNB', 'SOL', 'DOGE', 'ADA', 'TRX',
       'AVAX', 'LINK', 'WSTETH', 'TON', 'SUI', 'XLM', 'DOT'], dtype=object)

In [15]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume($),Symbol,Daily Return
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,BTC,NaN
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,BTC,-0.071926
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,BTC,-0.069843
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,BTC,0.035735
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,BTC,-0.024659


In [16]:
df.to_csv('data/top_crypto.csv', index=False)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35488 entries, 0 to 42910
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          35488 non-null  datetime64[ns]
 1   Open          35488 non-null  float64       
 2   High          35488 non-null  float64       
 3   Low           35488 non-null  float64       
 4   Close         35488 non-null  float64       
 5   Adj Close     35488 non-null  float64       
 6   Volume($)     35488 non-null  int64         
 7   Symbol        35488 non-null  object        
 8   Daily Return  35472 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 2.7+ MB


In [18]:
for x in range(2):
    print(x)

0
1
